# load data

In [10]:
import os
import numpy as np
from sklearn.model_selection import train_test_split

def load_data(root_path, test_size=0.2, random_state=42):
    """
    加载预处理的手势数据，并拆分为训练集和测试集。
    
    参数：
    root_path (str): 存储 processed_gesture_data.npy 和 gesture_labels.npy 的根目录
    test_size (float): 测试集比例，默认 20%
    random_state (int): 随机种子，确保可复现性
    
    返回：
    X_train, X_test, y_train, y_test: 训练集和测试集的数据及标签
    """
    data_path = os.path.join(root_path, "processed_gesture_data.npy")
    label_path = os.path.join(root_path, "gesture_labels.npy")
    
    if not os.path.exists(data_path) or not os.path.exists(label_path):
        raise FileNotFoundError("Processed gesture data or labels not found. Please run process_all_gesture_files first.")
    
    print("Loading processed data...")
    X = np.load(data_path,allow_pickle=True)
    y = np.load(label_path,allow_pickle=True)
    
    print(f"Data shape: {X.shape}, Labels shape: {y.shape}")
    
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size, random_state=random_state, stratify=y)
    
    print(f"Training set: {X_train.shape}, Testing set: {X_test.shape}")
    
    return X_train, X_test, y_train, y_test

data_folder = r"new_collect\fzh"  
X_train, X_test, y_train, y_test = load_data(data_folder)


Loading processed data...
Data shape: (325, 19, 100, 10), Labels shape: (325,)
Training set: (260, 19, 100, 10), Testing set: (65, 19, 100, 10)


# load selected data

In [8]:
import os
import numpy as np
from sklearn.model_selection import train_test_split

def load_data(root_path, selected_labels=None, test_size=0.2, random_state=42):
    """
    加载预处理的手势数据，并拆分为训练集和测试集，支持选择特定的标签。
    
    参数：
    - root_path (str): 存储 `processed_gesture_data.npy` 和 `gesture_labels.npy` 的根目录
    - selected_labels (list[int] or None): 选择加载的标签列表，例如 [1, 4, 6, 7]。如果 None，则加载所有数据。
    - test_size (float): 测试集比例，默认 20%
    - random_state (int): 随机种子，确保可复现性
    
    返回：
    - X_train, X_test, y_train, y_test: 训练集和测试集的数据及标签
    """
    data_path = os.path.join(root_path, "processed_gesture_data.npy")
    label_path = os.path.join(root_path, "gesture_labels.npy")
    
    if not os.path.exists(data_path) or not os.path.exists(label_path):
        raise FileNotFoundError("Processed gesture data or labels not found. Please run process_all_gesture_files first.")
    
    print("Loading processed data...")
    X = np.load(data_path, allow_pickle=True)
    y = np.load(label_path, allow_pickle=True)
    
    print(f"Original Data shape: {X.shape}, Labels shape: {y.shape}")

    # 选择指定标签的数据
    if selected_labels is not None:
        selected_labels = set(selected_labels)  # 转换为集合加速查询
        mask = np.isin(y, list(selected_labels))  # 选择符合标签的数据
        X = X[mask]
        y = y[mask]
        print(f"Filtered Data shape (after selecting labels {selected_labels}): {X.shape}")

    # 划分训练集和测试集
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=test_size, random_state=random_state, stratify=y
    )

    print(f"Training set: {X_train.shape}, Testing set: {X_test.shape}")
    
    return X_train, X_test, y_train, y_test

# 示例调用：加载所有数据
data_folder = r"new_collect\fzh"
X_train, X_test, y_train, y_test = load_data(data_folder)

# 示例调用：只加载选定的标签
selected_labels = [1, 4, 6, 7]
X_train, X_test, y_train, y_test = load_data(data_folder, selected_labels=selected_labels)


Loading processed data...
Original Data shape: (325, 19, 100, 10), Labels shape: (325,)
Training set: (260, 19, 100, 10), Testing set: (65, 19, 100, 10)
Loading processed data...
Original Data shape: (325, 19, 100, 10), Labels shape: (325,)
Filtered Data shape (after selecting labels {1, 4, 6, 7}): (100, 19, 100, 10)
Training set: (80, 19, 100, 10), Testing set: (20, 19, 100, 10)


# lstm

In [11]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout, TimeDistributed, Conv1D, Flatten
from tensorflow.keras.optimizers import Adam
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, MaxPooling1D, Flatten, Dense, Dropout, BatchNormalization, GlobalAveragePooling1D

X_train = np.nan_to_num(X_train, nan=0.0)
X_test = np.nan_to_num(X_test, nan=0.0)
y_train = np.nan_to_num(y_train, nan = 0.0)
y_test = np.nan_to_num(y_test, nan = 0.0)
# **获取数据形状**
num_batches = X_train.shape[0]  # batch 维度
num_windows = X_train.shape[1]  # 时间步（窗口数 59）
num_features = X_train.shape[2]  # 特征数（15）
num_channels = X_train.shape[3]  # 通道数（1）

# **检测类别数量**
unique_classes = np.unique(y_train)
num_classes = len(unique_classes)  # 确保类别数正确

print(f"Corrected Classes: {num_classes}, Batches: {num_batches}, Windows: {num_windows}, Features: {num_features}, Channels: {num_channels}")

# **保持 X 形状**
X_train = X_train.reshape(num_batches, num_windows, num_features, num_channels)
X_test = X_test.reshape(X_test.shape[0], num_windows, num_features, num_channels)
X_train = X_train[:,:,:,:]
X_test = X_test[:,:,:,:]

# print(X_train.shape)

num_features = X_train.shape[2] 
num_channels = X_train.shape[3]
# **标签编码**
encoder = LabelEncoder()
y_train = encoder.fit_transform(y_train)  # 转换成 0,1,2
y_test = encoder.transform(y_test)

# **转换为 One-Hot**
y_train = to_categorical(y_train, num_classes=num_classes)
y_test = to_categorical(y_test, num_classes=num_classes)

print(f"X_train shape: {X_train.shape}, y_train shape: {y_train.shape}")  # (batch, 59, 15, 1) (batch, 3)
print(f"X_test shape: {X_test.shape}, y_test shape: {y_test.shape}")  # (batch, 59, 15, 1) (batch, 3)


X_train = X_train.reshape(X_train.shape[0], X_train.shape[1], -1)  # (batch_size, 29, 40)
X_test = X_test.reshape(X_test.shape[0], X_test.shape[1], -1)  

# # print(num_features)
# # **构建 LSTM 处理通道的模型**

model = Sequential([
    # TimeDistributed(Conv1D(filters=16, kernel_size=3, activation='relu'), input_shape=(num_windows, num_features, num_channels)),  
    # TimeDistributed(GlobalAveragePooling1D()),  # 只对特征维度池化，不影响时间维度
    LSTM(64, return_sequences=True),  # 保持时间序列结构
    Dropout(0.2),
    LSTM(128, return_sequences=False),  # 输出 2D (batch, 128)
    Dense(64, activation='relu'),
    Dense(num_classes, activation='softmax')  # 最终分类
])

from tensorflow.keras.callbacks import LearningRateScheduler
def lr_schedule(epoch, lr):
    if epoch % 10 == 0 and epoch != 0:
        return lr * 0.8  # 每 10 个 epoch，学习率减半
    return lr

model.compile(
    optimizer=Adam(learning_rate=0.0001),  # 初始学习率
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

lr_callback = LearningRateScheduler(lr_schedule)

history = model.fit(X_train, y_train, epochs=100, callbacks=[lr_callback], validation_data=(X_test, y_test))

y_pred = model.predict(X_test)
y_pred_classes = np.argmax(y_pred, axis=1)
y_test_classes = np.argmax(y_test, axis=1)

accuracy = accuracy_score(y_test_classes, y_pred_classes)
print(f"Test Accuracy: {accuracy:.4f}")

# **保存模型**
# model.save("rnn_emg_model.h5")
# print("Model saved as rnn_emg_model.h5")


Corrected Classes: 13, Batches: 260, Windows: 19, Features: 100, Channels: 10
X_train shape: (260, 19, 100, 10), y_train shape: (260, 13)
X_test shape: (65, 19, 100, 10), y_test shape: (65, 13)
Epoch 1/100
9/9 [==============================] - 8s 210ms/step - loss: 2.5671 - accuracy: 0.0385 - val_loss: 2.5540 - val_accuracy: 0.1231 - lr: 1.0000e-04
Epoch 2/100
9/9 [==============================] - 0s 42ms/step - loss: 2.5492 - accuracy: 0.0769 - val_loss: 2.5331 - val_accuracy: 0.1077 - lr: 1.0000e-04
Epoch 3/100
9/9 [==============================] - 0s 42ms/step - loss: 2.5145 - accuracy: 0.1154 - val_loss: 2.5068 - val_accuracy: 0.1692 - lr: 1.0000e-04
Epoch 4/100
9/9 [==============================] - 0s 43ms/step - loss: 2.4954 - accuracy: 0.1423 - val_loss: 2.4919 - val_accuracy: 0.1538 - lr: 1.0000e-04
Epoch 5/100
9/9 [==============================] - 0s 42ms/step - loss: 2.4726 - accuracy: 0.2115 - val_loss: 2.4644 - val_accuracy: 0.2000 - lr: 1.0000e-04
Epoch 6/100
9/9 [===

In [12]:
from tensorflow.keras.callbacks import LearningRateScheduler
def lr_schedule(epoch, lr):
    if epoch % 10 == 0 and epoch != 0:
        return lr * 0.8  # 每 10 个 epoch，学习率减半
    return lr

# **编译模型**
model.compile(
    optimizer=Adam(learning_rate=0.0001),  # 初始学习率
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

# **使用回调函数**
lr_callback = LearningRateScheduler(lr_schedule)

# **训练时加上 callback**
history = model.fit(X_train, y_train, epochs=100, callbacks=[lr_callback], validation_data=(X_test, y_test))
y_pred = model.predict(X_test)
y_pred_classes = np.argmax(y_pred, axis=1)
y_test_classes = np.argmax(y_test, axis=1)

# **计算准确率**
accuracy = accuracy_score(y_test_classes, y_pred_classes)
print(f"Test Accuracy: {accuracy:.4f}")

Epoch 1/100
9/9 [==============================] - 8s 195ms/step - loss: 0.2855 - accuracy: 0.9346 - val_loss: 0.4091 - val_accuracy: 0.8769 - lr: 1.0000e-04
Epoch 2/100
9/9 [==============================] - 0s 35ms/step - loss: 0.2628 - accuracy: 0.9538 - val_loss: 0.4019 - val_accuracy: 0.8615 - lr: 1.0000e-04
Epoch 3/100
9/9 [==============================] - 0s 43ms/step - loss: 0.2716 - accuracy: 0.9308 - val_loss: 0.3932 - val_accuracy: 0.8615 - lr: 1.0000e-04
Epoch 4/100
9/9 [==============================] - 0s 33ms/step - loss: 0.2625 - accuracy: 0.9500 - val_loss: 0.3495 - val_accuracy: 0.9077 - lr: 1.0000e-04
Epoch 5/100
9/9 [==============================] - 0s 35ms/step - loss: 0.2716 - accuracy: 0.9231 - val_loss: 0.3516 - val_accuracy: 0.9231 - lr: 1.0000e-04
Epoch 6/100
9/9 [==============================] - 0s 33ms/step - loss: 0.2487 - accuracy: 0.9500 - val_loss: 0.3167 - val_accuracy: 0.9231 - lr: 1.0000e-04
Epoch 7/100
9/9 [==============================] - 0s 36m

# confusion matrix

In [ ]:
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix

# 计算混淆矩阵
cm = confusion_matrix(y_test_classes, y_pred_classes)

# 可视化
plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, fmt="d", cmap="Blues", xticklabels=encoder.classes_, yticklabels=encoder.classes_)
plt.xlabel("Predicted Label")
plt.ylabel("True Label")
plt.title("Confusion Matrix")
plt.show()


# history graph

In [ ]:
from matplotlib import pyplot as plt
plt.figure()
plt.plot(history.history["loss"])
plt.plot(history.history["val_loss"])
plt.show()